<a href="https://colab.research.google.com/github/annabellarinaldi/akaripuzzle/blob/main/8_Puzzle_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from heapq import heappush, heappop
from collections import deque
import time

class Node:
  def __init__(self, state, parent, depth):
    self.state = state
    self.parent = parent
    self.depth = depth

  def __lt__(self, other):
    return self.state < other.state

class PriorityQueue:
  def __init__(self):
    self.list = []

  def push(self, value, content):
    """ Add a node into the queue with priority value """
    # The middle value in the tuple is used as a tiebreaker
    # if the priority value is the same for multiple nodes.
    heappush(self.list, (value, len(self.list), content))

  def pop(self):
    """ Get the node with the smallest priority value """
    return heappop(self.list)[2]

  def __len__(self):
    """ Get the size of the queue. You can call len(queue) """
    return len(self.list)

class Searcher:
  def __init__(self):
    self.expanded_nodes_count = 0

  def search(self, start):
    pass

class State:
  def __init__(self, grid):
    self.grid = grid

  def __hash__(self):
    return hash((self.grid[0][0], self.grid[0][1], self.grid[0][2], \
                 self.grid[1][0], self.grid[1][1], self.grid[1][2], \
                 self.grid[2][0], self.grid[2][1], self.grid[2][2], ))

  def __eq__(self, other):
    return self.grid == other.grid


In [ ]:
def successors(state):
    i, j = find_empty_tile(state)
    moves = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    successors = []
    for di, dj in moves:
        ni, nj = i + di, j + dj
        if 0 <= ni < 3 and 0 <= nj < 3:
            new_state = [row[:] for row in state]
            new_state[i][j], new_state[ni][nj] = new_state[ni][nj], new_state[i][j]
            successors.append(new_state)
    return successors

GOAL = [[1,2,3],[4,5,6],[7,8,0]]

def find_empty_tile(state):
  for i in range(3):
      for j in range(3):
          if state[i][j] == 0:
              return i, j
  return None

def is_goal(state):
  return state == GOAL

In [ ]:
# Example test cases
print(successors([[1,3,6],[4,0,5],[7,8,2]]))
#  [[[1, 3, 6], [4, 8, 5], [7, 0, 2]], \
#  [[1, 3, 6], [4, 5, 0], [7, 8, 2]], \
#  [[1, 0, 6], [4, 3, 5], [7, 8, 2]], \
#  [[1, 3, 6], [0, 4, 5], [7, 8, 2]]]

print(successors([[7,1,2],[5,6,3],[0,4,8]]))
# [[[7, 1, 2], [5, 6, 3], [4, 0, 8]], [[7, 1, 2], [0, 6, 3], [5, 4, 8]]]

[[[1, 3, 6], [4, 8, 5], [7, 0, 2]], [[1, 3, 6], [4, 5, 0], [7, 8, 2]], [[1, 0, 6], [4, 3, 5], [7, 8, 2]], [[1, 3, 6], [0, 4, 5], [7, 8, 2]]]
[[[7, 1, 2], [5, 6, 3], [4, 0, 8]], [[7, 1, 2], [0, 6, 3], [5, 4, 8]]]


In [ ]:
from collections import deque

class BfsSearcher(Searcher):
    def search(self, start):
        queue = deque([Node(start, None, 0)])
        visited = set([State(start)])
        while queue:
            node = queue.popleft()
            state_hash = State(node.state)

            if node.state == GOAL:
                return node

            self.expanded_nodes_count += 1

            for s in successors(node.state):
                new_state_hash = State(s)
                if new_state_hash not in visited:
                    visited.add(new_state_hash)
                    queue.append(Node(s, node, node.depth + 1))

        return None

In [ ]:
import time

start_easy = [[1,2,3],[5,0,7],[4,8,6]]

start_hard = [[1,2,3],[4,7,6],[0,8,5]]


for i, state in enumerate([start_easy, start_hard]):
  print(f"=== State {i} ===")
  t = time.time()
  searcher = BfsSearcher()
  sol = searcher.search(state)
  elapsed_time = time.time() - t
  print(sol.state)
  print("Elapsed time for BFS:", elapsed_time)
  print("Expanded nodes for BFS:", searcher.expanded_nodes_count)
  print("Depth for BFS:", sol.depth)


=== State 0 ===
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
Elapsed time for BFS: 0.01513218879699707
Expanded nodes for BFS: 243
Depth for BFS: 8
=== State 1 ===
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
Elapsed time for BFS: 0.4574923515319824
Expanded nodes for BFS: 7572
Depth for BFS: 16


In [ ]:
class IdsSearcher(Searcher):
  def __init__(self, max_depth=100):
    super().__init__()
    self.max_depth = max_depth
  def depth_limited_search(self, node, depth_limit, visited):
        if node.state == GOAL:
            return node
        if node.depth >= depth_limit:
            return None
        state_hash = State(node.state)
        if state_hash not in visited or visited[state_hash] > node.depth:
            visited[state_hash] = node.depth
            self.expanded_nodes_count += 1
            for s in successors(node.state):
              child_node = Node(s, node, node.depth + 1)
              result = self.depth_limited_search(child_node, depth_limit, visited)
              if result:
                  return result
        return None
  def search(self, start):
    depth = 0
    while depth < self.max_depth:
        visited = {}
        start_node = Node(start, None, 0)
        result = self.depth_limited_search(start_node, depth, visited)
        if result:
            return result
        depth += 1
    return None

In [ ]:
# Run the algorithm
start_easy = [[1, 2, 3], [5, 0, 7], [4, 8, 6]]
start_hard = [[1, 2, 3], [4, 7, 6], [0, 8, 5]]

for i, state in enumerate([start_easy, start_hard]):
  print(f"=== State {i} ===")
  t = time.time()
  searcher = IdsSearcher()
  sol = searcher.search(state)
  elapsed_time = time.time() - t
  print(sol.state)
  print("Elapsed time for IDS:", elapsed_time)
  print("Expanded nodes for IDS:", searcher.expanded_nodes_count)
  print("Depth for IDS:", sol.depth)

=== State 0 ===
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
Elapsed time for IDS: 0.006863117218017578
Expanded nodes for IDS: 344
Depth for IDS: 8
=== State 1 ===
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
Elapsed time for IDS: 0.37500929832458496
Expanded nodes for IDS: 14559
Depth for IDS: 16


In [ ]:
def manhattan_heuristic(node):
    state = node.state if hasattr(node, 'state') else node
    distance = 0
    for i in range(3):
        for j in range(3):
            value = state[i][j]
            if value != 0:
                goal_x, goal_y = (value - 1) // 3, (value - 1) % 3
                distance += abs(i - goal_x) + abs(j - goal_y)
    return distance

def misplaced_tiles_heuristic(node):
    state = node.state if hasattr(node, 'state') else node
    return sum(1 for i in range(3) for j in range(3) if state[i][j] != 0 and state[i][j] != GOAL[i][j])

class AStarSearcher(Searcher):
    def search(self, start, heuristic):
        pq = PriorityQueue()
        start_node = Node(start, None, 0)
        pq.push(heuristic(start_node), start_node)
        visited = {}

        while pq:
            node = pq.pop()

            # Correct goal check
            if node.state == GOAL:
                return node

            state_hash = State(node.state)
            g_cost = node.depth
            if state_hash in visited and visited[state_hash] <= node.depth:
                continue

            visited[state_hash] = g_cost
            self.expanded_nodes_count += 1

            for s in successors(node.state):
                new_node = Node(s, node, node.depth + 1)
                f_cost = new_node.depth + heuristic(new_node)
                pq.push(f_cost, new_node)

        return None


In [ ]:
# Example test cases
print(manhattan_heuristic([[7,1,2],[5,6,3],[0,4,8]]))
# should be 10

print(misplaced_tiles_heuristic([[1,2,3],[0,5,6],[4,7,8]]))
# should be 3

10
3


In [ ]:
for i, state in enumerate([start_easy, start_hard]):
  print(f"=== State {i} ===")

  t = time.time()
  searcher = AStarSearcher()
  sol = searcher.search(state, manhattan_heuristic)
  elapsed_time = time.time() - t
  print("Elapsed time for A star with Manhattan:", elapsed_time)
  print("Expanded nodes for A star with Manhattan:", searcher.expanded_nodes_count)
  print(sol.state)

  t = time.time()
  searcher = AStarSearcher()
  sol = searcher.search(state, misplaced_tiles_heuristic)
  elapsed_time = time.time() - t
  print("Elapsed time for A star with Misplaced Tiles:", elapsed_time)
  print("Expanded nodes for A star with Misplaced Tiles:", searcher.expanded_nodes_count)
  print(sol.state)

=== State 0 ===
Elapsed time for A star with Manhattan: 0.0005214214324951172
Expanded nodes for A star with Manhattan: 15
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
Elapsed time for A star with Misplaced Tiles: 0.0006506443023681641
Expanded nodes for A star with Misplaced Tiles: 26
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
=== State 1 ===
Elapsed time for A star with Manhattan: 0.028873682022094727
Expanded nodes for A star with Manhattan: 259
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
Elapsed time for A star with Misplaced Tiles: 0.0661165714263916
Expanded nodes for A star with Misplaced Tiles: 663
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
